<a href="https://colab.research.google.com/github/MateusMendes535/P2---Inteligencia-Artificial---Dimmy---ICEV/blob/main/P2_Inteligencia_Artificial_Dimmy_ICEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install scikit-learn

In [ ]:
!pip install torch

In [ ]:
!pip install -U accelerate

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Carregar o texto longo
with open('/Compêndio-Eleitoral_2024_Legislação-Eleitoral-e-Normativos-TREPR.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Função para dividir o texto em chunks de até 512 tokens
def split_long_text(text, tokenizer, max_length=512):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        token_length = len(tokenizer.tokenize(word))
        if current_length + token_length > max_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = token_length
        else:
            current_chunk.append(word)
            current_length += token_length

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Tokenizer para DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
chunks = split_long_text(text, tokenizer)

# Criar um dataset com os chunks e labels fictícias para exemplo
dataset = Dataset.from_dict({
    'text': chunks,
    'label': [0]*len(chunks)  # Aqui você deve colocar suas labels reais
})

# Função de tokenização para o dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
# Carregar o modelo DistilBERT para classificação
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Configurações de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Ajuste conforme a memória disponível
    num_train_epochs=3,
    weight_decay=0.01,
)

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Treinar o modelo
trainer.train()


In [ ]:
# Avaliar o modelo
results = trainer.evaluate()
print(results)

# Salvar o modelo fine-tuned
model.save_pretrained("path/to/save/model")
tokenizer.save_pretrained("path/to/save/model")


In [ ]:
from transformers import pipeline

# Carregar o modelo fine-tuned
model = DistilBertForSequenceClassification.from_pretrained("path/to/save/model")
tokenizer = DistilBertTokenizer.from_pretrained("path/to/save/model")

# Criar pipeline de classificação
nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Exemplo de uso
result = nlp("Exemplo de texto sobre direito eleitoral")
print(result)
